In [29]:
import pandas as pd
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")
train1 = train.drop(['Descript', 'Resolution'], axis = 1)

In [30]:
train1 = train1.query('Y != 90')

In [31]:
Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
def to_season(month_num):
    season = 'winter'
    if 3<=month_num<=5:
        season = 'spring'
    elif 6<=month_num<=8:
        season = 'summer'
    elif 9<=month_num<=11:
        season = 'autumn'
    return season
train1['Season'] =Dt.dt.month.apply(to_season)
train1.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

count_encoding = train1['PdDistrict'].value_counts().to_dict()
train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)

train1['Category'] = le.fit_transform(train1['Category'])

train1.head()
# DayOfWeek...label,PdDistrict...count

,Category,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour,Season
0,37,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,spring
1,21,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,spring
2,21,6,105284,0,-122.424363,37.800414,2015-05-13,2015,5,13,23,spring
3,16,6,105284,1,-122.426995,37.800873,2015-05-13,2015,5,13,23,spring
4,16,6,49311,1,-122.438738,37.771541,2015-05-13,2015,5,13,23,spring


In [32]:
train1_SS = pd.get_dummies(train1['Season'])

train1 = pd.concat([train1, train1_SS], axis = 1)

train1.drop(['Season'], axis = 1, inplace=True)
train1.head()

,Category,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour,autumn,spring,summer,winter
0,37,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,0,1,0,0
1,21,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,0,1,0,0
2,21,6,105284,0,-122.424363,37.800414,2015-05-13,2015,5,13,23,0,1,0,0
3,16,6,105284,1,-122.426995,37.800873,2015-05-13,2015,5,13,23,0,1,0,0
4,16,6,49311,1,-122.438738,37.771541,2015-05-13,2015,5,13,23,0,1,0,0


In [33]:
Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
def to_season(month_num):
    season = 'winter'
    if 3<=month_num<=5:
        season = 'spring'
    elif 6<=month_num<=8:
        season = 'summer'
    elif 9<=month_num<=11:
        season = 'autumn'
    return season
test['Season'] =Dt.dt.month.apply(to_season)
test.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
test['DayOfWeek'] = le.fit_transform(test['DayOfWeek'])

count_encoding = test['PdDistrict'].value_counts().to_dict()
test['PdDistrict'] = test['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)
test.head()

,Id,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour,Season
0,0,3,89591,1,-122.399588,37.735051,2015-05-10,2015,5,10,23,spring
1,1,3,89591,0,-122.391523,37.732432,2015-05-10,2015,5,10,23,spring
2,2,3,107017,1,-122.426002,37.792212,2015-05-10,2015,5,10,23,spring
3,3,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,spring
4,4,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,spring


In [34]:
test_SS = pd.get_dummies(test['Season'])

test = pd.concat([test, test_SS], axis = 1)

test.drop(['Season'], axis = 1, inplace=True)
test.head()

,Id,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour,autumn,spring,summer,winter
0,0,3,89591,1,-122.399588,37.735051,2015-05-10,2015,5,10,23,0,1,0,0
1,1,3,89591,0,-122.391523,37.732432,2015-05-10,2015,5,10,23,0,1,0,0
2,2,3,107017,1,-122.426002,37.792212,2015-05-10,2015,5,10,23,0,1,0,0
3,3,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,0,1,0,0
4,4,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,0,1,0,0


In [35]:
cols = ['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour',"spring","summer","autumn","winter"]
X = train1[cols]
Y = train1['Category']
X_test = test[cols]

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [36]:
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

In [37]:
accuracy_score(y_val, y_pred)

0.2910229882875529

In [38]:
xgb_submission = model.predict(X_test)

In [40]:
import csv
pred = model.predict_proba(test[['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour',"spring","summer","autumn","winter"]])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)
submission.head()

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.003844,0.140790,0.000066,0.000319,0.027236,0.001178,0.005635,0.011154,0.001876,...,0.000318,0.008045,0.000558,0.031212,1.770239e-06,0.006418,0.068538,0.180563,0.035079,0.029997
1,1,0.001809,0.063849,0.000020,0.000839,0.001859,0.000521,0.012064,0.029174,0.001986,...,0.000166,0.005566,0.003807,0.017856,1.218910e-05,0.000365,0.018800,0.059674,0.092005,0.035548
2,2,0.002623,0.048917,0.000036,0.000041,0.152745,0.000617,0.000692,0.005509,0.002827,...,0.000004,0.006839,0.000118,0.022208,2.004294e-07,0.010689,0.052836,0.142156,0.010387,0.002207
3,3,0.001251,0.126138,0.000012,0.001115,0.029827,0.002587,0.002643,0.012804,0.004337,...,0.000200,0.007190,0.000137,0.032344,8.104988e-07,0.010033,0.069611,0.157514,0.025124,0.018304
4,4,0.001251,0.126138,0.000012,0.001115,0.029827,0.002587,0.002643,0.012804,0.004337,...,0.000200,0.007190,0.000137,0.032344,8.104988e-07,0.010033,0.069611,0.157514,0.025124,0.018304


In [41]:
submission.to_csv('XGB_soh.csv',index = False)